In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#import Plotly 
import plotly.express as px

In [3]:
#print head of cleaned dataset
DatabaseDF = pd.read_csv("../CleanedDataset.csv")
DatabaseDF.head()

,Unnamed: 0,Operator,Well Name,Formation,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs",...,Fluid Type from DI,SPACING_CAPPED,12 month Cum Prod,Completion Type,Sleeves,P&P,Frac Ports,Repeater Ports,CT,Year Drilled
0,0,Operator 1,Well 1,MBH,5114.0,30.0,141,99,10,0.0,...,NaN,660.0,33.55,NaN,NaN,NaN,NaN,NaN,NaN,2006.0
1,1,Operator 2,Well 2,MBH,5181.0,NaN,142,99,16,418000.0,...,NaN,660.0,27.62,NaN,NaN,NaN,NaN,NaN,NaN,2005.0
2,2,Operator 2,Well 3,MBH,4780.0,15.0,142,99,24,568407.0,...,NaN,660.0,11.19,OH,NaN,NaN,NaN,NaN,NaN,2006.0
3,3,Operator 1,Well 4,MBH,10654.0,1.0,142,99,1,845000.0,...,NaN,660.0,27.85,NaN,NaN,NaN,NaN,NaN,NaN,2007.0
4,4,Operator 2,Well 5,MBH,5209.0,1.0,142,99,35,297366.0,...,NaN,660.0,50.88,OH,NaN,NaN,NaN,NaN,NaN,2006.0


# Spider (Radar) Chart: Definition

A radar chart is used primarily in fields such as Business to compare various quantitative variables in a dataset. The chart has multiple radii, where each radius represents one variable. The magnitude of the variable is represented by the length of the radius. The longer the radius, the more of an effect the variable has. In this notebook specifically, a longer radius represents a variable more correlated to the cumulative production.

The Radar Chart is a useful tool to visualize how the features in our dataset affect each other and the oil production, and will help us with feature selection because we can narrow down the variables and focus on the ones that seem the most impactful. 

This notebook will use different correlation coefficients, such as Pearson, Spearman's and Kendall, to analyze variables with the radar charts.

## Pearson's Correlation

In [111]:
#Finding Pearson's correlation coefficient for each feature
pearson_correlation = DatabaseDF.corr(method = 'pearson')
pearson_correlation

,Unnamed: 0,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Best1 Mo BOPD,Best3 Mo BOPD,...,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod,Sleeves,P&P,Frac Ports,Repeater Ports,CT,Year Drilled
Unnamed: 0,1.000000,0.053661,0.228829,0.243115,0.011720,-0.050113,0.203161,0.213521,0.094560,0.096491,...,0.245966,-0.052245,-0.107073,0.119804,0.186390,0.173148,-0.711513,0.535451,0.191782,0.186718
Lateral Length,0.053661,1.000000,0.456774,-0.038726,0.241831,-0.021131,0.253713,0.256175,0.113087,0.113343,...,-0.037672,-0.233496,-0.147507,0.147342,0.391450,0.528658,-0.087372,0.349897,0.391838,0.313809
Stages,0.228829,0.456774,1.000000,0.040092,0.147589,0.024033,0.587432,0.520606,0.265778,0.263466,...,0.038070,-0.154342,-0.245130,0.344700,0.894700,0.880684,0.193914,0.078326,0.983473,0.718240
Township,0.243115,-0.038726,0.040092,1.000000,-0.011333,0.023008,-0.005835,-0.006636,-0.121909,-0.110662,...,0.998011,-0.145898,0.014882,-0.114935,0.061510,-0.008466,-0.756806,0.716801,0.011609,-0.001583
Range,0.011720,0.241831,0.147589,-0.011333,1.000000,-0.019859,-0.021746,0.194991,-0.164083,-0.192811,...,-0.009201,-0.983505,0.009955,-0.194452,0.112901,0.133063,0.602259,0.045222,-0.185800,0.164663
Section,-0.050113,-0.021131,0.024033,0.023008,-0.019859,1.000000,0.020073,0.017878,0.010863,0.012004,...,-0.037652,0.016787,-0.010710,0.004276,0.033284,0.008336,0.459620,-0.436849,0.098984,0.029651
"Total Prop, lbs",0.203161,0.253713,0.587432,-0.005835,-0.021746,0.020073,1.000000,0.714701,0.371468,0.408457,...,-0.007611,0.020765,-0.161274,0.480937,0.501151,0.553368,0.469282,0.004144,-0.487679,0.535724
"Fluid, bbls",0.213521,0.256175,0.520606,-0.006636,0.194991,0.017878,0.714701,1.000000,0.241188,0.260443,...,-0.008240,-0.194170,-0.166497,0.392504,0.346689,0.542364,0.579526,0.173785,-0.499150,0.538226
Best1 Mo BOPD,0.094560,0.113087,0.265778,-0.121909,-0.164083,0.010863,0.371468,0.241188,1.000000,0.964461,...,-0.123385,0.181494,-0.131038,0.874867,0.196826,0.206115,0.328419,-0.593844,0.445904,0.193007
Best3 Mo BOPD,0.096491,0.113343,0.263466,-0.110662,-0.192811,0.012004,0.408457,0.260443,0.964461,1.000000,...,-0.112215,0.208846,-0.119870,0.936788,0.171387,0.227400,0.299310,-0.749003,0.412009,0.191037


In [112]:
#Printing the correlation to Cumulative Production specifically
pearson_correlation['12 month Cum Prod']

Unnamed: 0           0.119804
Lateral Length       0.147342
Stages               0.344700
Township            -0.114935
Range               -0.194452
Section              0.004276
Total Prop, lbs      0.480937
Fluid, bbls          0.392504
Best1 Mo BOPD        0.874867
Best3 Mo BOPD        0.936788
Best6 Mo BOPD        0.973459
Best9 Mo BOPD        0.983770
Best12 Mo BOPD       0.984430
Latitude            -0.115832
Longitude            0.209950
SPACING_CAPPED      -0.137795
12 month Cum Prod    1.000000
Sleeves              0.216394
P&P                  0.318438
Frac Ports           0.333429
Repeater Ports      -0.787614
CT                   0.334719
Year Drilled         0.286433
Name: 12 month Cum Prod, dtype: float64

### First set of variables:

In [113]:
#Features that are being compared
features = ['Lateral Length', 'Stages', 'Fluid, bbls', 'Total Prop, lbs', 'Longitude']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(pearson_correlation.at[i, '12 month Cum Prod'])

        '''I would explain why this code is being done. and why Pearson specifically.'''


In [114]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

### Second set of variables

In [115]:
#Features that are being compared
features = ['Latitude','CT','Section',
           'Year Drilled', 'SPACING_CAPPED']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(pearson_correlation.at[i, '12 month Cum Prod'])

In [116]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

Pearson correlation shows that Completion Type, Total Proppant, and Fluid seem to have the highest correlation with cumulative production, not including the BOPD values for every third month. 

## Spearman's Rank Correlation

In [117]:
#Spearman's Rank correlation coefficient for each feature
spearman_correlation = DatabaseDF.corr(method = 'spearman')
spearman_correlation

,Unnamed: 0,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Best1 Mo BOPD,Best3 Mo BOPD,...,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod,Sleeves,P&P,Frac Ports,Repeater Ports,CT,Year Drilled
Unnamed: 0,1.000000,0.077660,0.232800,0.381223,0.006677,-0.049775,0.243052,0.198040,0.127087,0.129392,...,0.385105,-0.044463,-0.118379,0.145759,0.203367,0.178904,-0.575608,0.666886,0.176944,0.184971
Lateral Length,0.077660,1.000000,0.435665,0.018391,0.181624,-0.030477,0.326196,0.307917,0.141420,0.142319,...,0.021630,-0.185652,-0.184828,0.177257,0.405629,0.424934,0.028158,0.872082,0.480787,0.312223
Stages,0.232800,0.435665,1.000000,0.052379,0.162301,0.029076,0.658696,0.630845,0.251405,0.244439,...,0.050454,-0.171085,-0.291524,0.340166,0.908350,0.877969,0.301994,0.229416,1.000000,0.648229
Township,0.381223,0.018391,0.052379,1.000000,-0.018511,0.025329,0.010163,-0.021137,-0.174974,-0.165998,...,0.997491,-0.123387,0.036315,-0.165085,0.104709,0.022196,-0.688001,0.666886,-0.172964,-0.039493
Range,0.006677,0.181624,0.162301,-0.018511,1.000000,-0.020666,0.133176,0.274213,-0.121818,-0.144633,...,-0.014452,-0.983496,-0.020250,-0.157996,0.138849,0.162309,0.449071,0.648886,0.164849,0.167011
Section,-0.049775,-0.030477,0.029076,0.025329,-0.020666,1.000000,0.025029,0.021621,0.018478,0.016098,...,-0.036515,0.017142,-0.017271,0.010521,0.039962,0.009927,0.319731,-0.205196,0.008116,0.032135
"Total Prop, lbs",0.243052,0.326196,0.658696,0.010163,0.133176,0.025029,1.000000,0.856374,0.320731,0.331295,...,0.008453,-0.136473,-0.279404,0.440060,0.529296,0.615773,0.424671,0.564288,-0.151921,0.611360
"Fluid, bbls",0.198040,0.307917,0.630845,-0.021137,0.274213,0.021621,0.856374,1.000000,0.282209,0.291956,...,-0.022095,-0.272063,-0.289765,0.401960,0.507377,0.629173,0.680200,-0.205196,-0.066131,0.649717
Best1 Mo BOPD,0.127087,0.141420,0.251405,-0.174974,-0.121818,0.018478,0.320731,0.282209,1.000000,0.970644,...,-0.177390,0.139445,-0.220560,0.891192,0.184210,0.188430,0.204660,-0.820783,0.495085,0.206711
Best3 Mo BOPD,0.129392,0.142319,0.244439,-0.165998,-0.144633,0.016098,0.331295,0.291956,0.970644,1.000000,...,-0.168382,0.161616,-0.225671,0.937187,0.160880,0.202089,0.214894,-0.666886,0.443253,0.206136


In [118]:
#Printing the correlation to Cumulative Production specifically
spearman_correlation['12 month Cum Prod']

Unnamed: 0           0.145759
Lateral Length       0.177257
Stages               0.340166
Township            -0.165085
Range               -0.157996
Section              0.010521
Total Prop, lbs      0.440060
Fluid, bbls          0.401960
Best1 Mo BOPD        0.891192
Best3 Mo BOPD        0.937187
Best6 Mo BOPD        0.968343
Best9 Mo BOPD        0.977030
Best12 Mo BOPD       0.977382
Latitude            -0.166509
Longitude            0.175363
SPACING_CAPPED      -0.246285
12 month Cum Prod    1.000000
Sleeves              0.192228
P&P                  0.328136
Frac Ports           0.211056
Repeater Ports      -0.974679
CT                   0.471850
Year Drilled         0.311370
Name: 12 month Cum Prod, dtype: float64

### First set of variables

In [119]:
#Features that are being compared
features = ['Lateral Length', 'Stages', 'Fluid, bbls', 'Total Prop, lbs', 'Longitude']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [120]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

### Second set of variables

In [121]:
#Features that are being compared
features = ['Latitude','CT','Section',
           'Year Drilled', 'SPACING_CAPPED']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(spearman_correlation.at[i, '12 month Cum Prod'])

In [122]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

Spearman's coefficient shows the same as the Pearson coefficient: Completion Type, Total Proppant, and Fluid. It also might be worth looking more into the number of stages and the year each well is drilled.

## Kendall Correlation

In [123]:
#finding Pearson's correlation coefficient for each feature
kendall_correlation = DatabaseDF.corr(method = 'kendall')
kendall_correlation

,Unnamed: 0,Lateral Length,Stages,Township,Range,Section,"Total Prop, lbs","Fluid, bbls",Best1 Mo BOPD,Best3 Mo BOPD,...,Latitude,Longitude,SPACING_CAPPED,12 month Cum Prod,Sleeves,P&P,Frac Ports,Repeater Ports,CT,Year Drilled
Unnamed: 0,1.000000,0.052686,0.168437,0.358195,0.004109,-0.033591,0.167874,0.141531,0.084968,0.086246,...,0.352100,-0.028989,-0.090092,0.097114,0.151749,0.125591,-0.462069,0.527046,0.153112,0.160334
Lateral Length,0.052686,1.000000,0.309936,0.012854,0.125361,-0.020888,0.222342,0.207519,0.095635,0.096264,...,0.014574,-0.123547,-0.135229,0.120099,0.286365,0.303404,0.024845,0.737865,0.382780,0.226446
Stages,0.168437,0.309936,1.000000,0.037471,0.115880,0.020303,0.502807,0.471214,0.177771,0.172709,...,0.035094,-0.116569,-0.224938,0.242970,0.886579,0.876172,0.234765,0.125988,1.000000,0.522863
Township,0.358195,0.012854,0.037471,1.000000,-0.016345,0.017285,0.006452,-0.013354,-0.122942,-0.117739,...,0.967689,-0.081553,0.026258,-0.119089,0.074185,0.013643,-0.564815,0.527046,-0.156795,-0.028170
Range,0.004109,0.125361,0.115880,-0.016345,1.000000,-0.014719,0.094244,0.193712,-0.085576,-0.101709,...,-0.012843,-0.913127,-0.014247,-0.110660,0.094662,0.121660,0.344016,0.471405,0.198583,0.120057
Section,-0.033591,-0.020888,0.020303,0.017285,-0.014719,1.000000,0.017026,0.014576,0.012530,0.010970,...,-0.036827,0.011810,-0.012705,0.007134,0.028297,0.006999,0.250313,-0.316228,0.019913,0.023110
"Total Prop, lbs",0.167874,0.222342,0.502807,0.006452,0.094244,0.017026,1.000000,0.671173,0.219196,0.227002,...,0.005248,-0.092848,-0.206224,0.307798,0.391270,0.469789,0.346552,0.316228,-0.114834,0.466560
"Fluid, bbls",0.141531,0.207519,0.471214,-0.013354,0.193712,0.014576,0.671173,1.000000,0.193036,0.200793,...,-0.013505,-0.185686,-0.213262,0.281894,0.378290,0.471592,0.573886,-0.105409,-0.019139,0.498299
Best1 Mo BOPD,0.084968,0.095635,0.177771,-0.122942,-0.085576,0.012530,0.219196,0.193036,1.000000,0.857700,...,-0.121036,0.094545,-0.164380,0.726456,0.129416,0.133937,0.165025,-0.737865,0.382780,0.150646
Best3 Mo BOPD,0.086246,0.096264,0.172709,-0.117739,-0.101709,0.010970,0.227002,0.200793,0.857700,1.000000,...,-0.116019,0.109660,-0.167810,0.810152,0.112321,0.144026,0.165025,-0.527046,0.344502,0.150682


In [124]:
#Printing the correlation to Cumulative Production specifically
kendall_correlation['12 month Cum Prod']

Unnamed: 0           0.097114
Lateral Length       0.120099
Stages               0.242970
Township            -0.119089
Range               -0.110660
Section              0.007134
Total Prop, lbs      0.307798
Fluid, bbls          0.281894
Best1 Mo BOPD        0.726456
Best3 Mo BOPD        0.810152
Best6 Mo BOPD        0.878590
Best9 Mo BOPD        0.909820
Best12 Mo BOPD       0.915567
Latitude            -0.116349
Longitude            0.118575
SPACING_CAPPED      -0.180903
12 month Cum Prod    1.000000
Sleeves              0.134486
P&P                  0.235428
Frac Ports           0.165025
Repeater Ports      -0.948683
CT                   0.382780
Year Drilled         0.228674
Name: 12 month Cum Prod, dtype: float64

### First set of variables

In [125]:
#Features that are being compared
features = ['Lateral Length', 'Stages', 'Fluid, bbls', 'Total Prop, lbs', 'Longitude']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(kendall_correlation.at[i, '12 month Cum Prod'])

In [126]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

### Second set of variables

In [127]:
#Features that are being compared
features = ['Latitude','CT','Section',
           'Year Drilled', 'SPACING_CAPPED']
#Radius values for each variable
radius_values = []
for i in features:
        radius_values.append(kendall_correlation.at[i, '12 month Cum Prod'])

In [128]:
#create dictionary for angle and radius values 
df = pd.DataFrame(dict(
    r= radius_values,
    theta= features))

#create radar chart
fig = px.line_polar(df, r='r', theta='theta', line_close=True)

#fill in area
fig.update_traces(fill='toself')

#plot chart
fig.show()

Kendall's coefficient shows the same features that have the highest correlation coefficient. 

According to the radar charts created for the three different correlation coefficients, it seems that Completion Type, Total Proppant, and Fluid seem to be the three quantitative variables that affect the cumulative production the most. However, these radar charts do not include the categorical variables in the dataset.

In [ ]:
''' I really like this model. Primarily because this allows a really good comparison. If you want the radar chart to compare categorical variable in the dataset, I would change the categorical data to numerical values. 

Here is an example:

labels = DatabaseDF['variable'].unique().tolist()
mapping = dict( zip(labels,range(len(labels))) )
DatabaseDF.replace({'variable': mapping},inplace=True)